## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-09-19-48-23 +0000,nyt,Former President Yoon Suk Yeol of South Korea ...,https://www.nytimes.com/2025/07/09/world/asia/...
1,2025-07-09-19-47-40 +0000,nyt,Can Canada Leverage Its Unique U.S. Relationsh...,https://www.nytimes.com/2025/07/09/world/canad...
2,2025-07-09-19-47-26 +0000,bbc,Linda Yaccarino departs as boss of Musk's X,https://www.bbc.com/news/articles/cx2gy3j9xq6o
3,2025-07-09-19-46-16 +0000,bbc,Gaza truce talks reportedly stall despite seco...,https://www.bbc.com/news/articles/ce3newwl1zeo
4,2025-07-09-19-12-20 +0000,nyt,A Surgeon Shares What She Saw in Gaza’s Hospitals,https://www.nytimes.com/2025/07/09/world/middl...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2298/2741963311.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
17,trump,17
8,new,5
42,tariffs,5
66,ukraine,4
82,texas,4


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
21,2025-07-09-16-01-38 +0000,nyt,Trump to Discuss Economic Investment With Afri...,https://www.nytimes.com/2025/07/09/us/politics...,32
9,2025-07-09-18-42-11 +0000,nyt,Russia Makes Record Attack on Ukraine as Trump...,https://www.nytimes.com/2025/07/09/world/europ...,32
6,2025-07-09-19-05-45 +0000,nyt,Trump Tariffs Aim to Settle Scores With Countr...,https://www.nytimes.com/2025/07/09/business/ec...,31
1,2025-07-09-19-47-40 +0000,nyt,Can Canada Leverage Its Unique U.S. Relationsh...,https://www.nytimes.com/2025/07/09/world/canad...,30
66,2025-07-08-23-01-40 +0000,bbc,Faisal Islam: Trump delays tariffs as the rest...,https://www.bbc.com/news/articles/c628nk51364o,30


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
21,32,2025-07-09-16-01-38 +0000,nyt,Trump to Discuss Economic Investment With Afri...,https://www.nytimes.com/2025/07/09/us/politics...
43,20,2025-07-09-09-32-29 +0000,bbc,At least three dead in New Mexico flash flooding,https://www.bbc.com/news/articles/cx2jk51xv65o
29,19,2025-07-09-13-21-24 +0000,nyt,European Court Holds Russia Liable for Human R...,https://www.nytimes.com/2025/07/09/world/europ...
20,15,2025-07-09-16-06-35 +0000,bbc,EU hopes to agree US tariff deal 'in coming days',https://www.bbc.com/news/articles/cev0n01y8wwo
36,15,2025-07-09-12-22-42 +0000,bbc,"'Hero' dad, twin girls and riverside campers a...",https://www.bbc.com/news/articles/c5ygl8lpyyqo
6,14,2025-07-09-19-05-45 +0000,nyt,Trump Tariffs Aim to Settle Scores With Countr...,https://www.nytimes.com/2025/07/09/business/ec...
70,14,2025-07-08-21-30-30 +0000,cbc,Building Canada Act a 'troubling threat' to In...,https://www.cbc.ca/news/indigenous/bill-c5-amn...
0,13,2025-07-09-19-48-23 +0000,nyt,Former President Yoon Suk Yeol of South Korea ...,https://www.nytimes.com/2025/07/09/world/asia/...
17,12,2025-07-09-17-23-34 +0000,nyt,"‘Ne Zha 2,’ Blockbuster Chinese Animated Film,...",https://www.nytimes.com/2025/07/09/movies/ne-z...
28,12,2025-07-09-13-33-22 +0000,nyt,"Not Invited to Dior’s Show in Paris, a Fashion...",https://www.nytimes.com/2025/07/09/style/dior-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
